In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import random
from collections import defaultdict

# All yoga channels URLs to a list

In [43]:
chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

In [44]:
query = 'yoga'
youtube_search = 'https://www.youtube.com/results?search_query='
channel_filter = '&sp=EgIQAg%253D%253D'
youtube_query = youtube_search + query.replace(' ', '+') + channel_filter
youtube_query

'https://www.youtube.com/results?search_query=yoga&sp=EgIQAg%253D%253D'

In [45]:
driver = webdriver.Chrome(chromedriver)
driver.get(youtube_query)

In [46]:
#Scroll until len(yoga_channel) is max
for i in range(40):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1)

In [47]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
contents_div = soup.find('div', id='contents')

yoga_channel = []
for title in contents_div.find_all('a', id='main-link', attrs={'class': 'channel-link yt-simple-endpoint style-scope ytd-channel-renderer'}):
    channel_url = f"https://www.youtube.com{title['href']}"
    yoga_channel.append(channel_url)

In [48]:
len(yoga_channel)

607

In [49]:
with open('Attempt2/yoga_channel_607_att2.pkl', 'wb') as picklefile:
    pickle.dump(yoga_channel, picklefile)

## Scraping "About" page of yoga channels

In [56]:
def get_channel_info_about(url):
    
    '''
    Scrape data from 'About' section of YouTube channel.
    ---
    Input: url (string)
    Output: dictionary
    
    '''
    channel_data = {}
    try:
        response = requests.get((url + '/about'))
        page = response.text
        soup = BeautifulSoup(page, "html5lib")
        contents_div = soup.find('div', id='content')
    except:
        channel_data = {} 
    try:
        channel_data['name'] = contents_div.find('span', attrs={'class': 'about-channel-link-text'}).text.strip()
    except:
        channel_data['name'] = ''
    try:
        channel_data['total_views'] = contents_div.find('span', attrs={'class': 'about-stat'}).find('b').text.strip()
    except:
        channel_data['total_views'] = '' 
    try:
        channel_data['joined_date'] = contents_div.find_all('span', attrs={'class': 'about-stat'})[1].text.strip()
    except:
        channel_data['joined_date'] = ''   
    try:
        channel_data['subscriber_count'] = contents_div.find('span', attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal subscribed yt-uix-tooltip'}).text.strip()
    except:
        channel_data['subscriber_count'] = ''   
    try:
        channel_data['n_featured_channels'] = len(contents_div.find_all('h3', attrs={'class': 'yt-lockup-title'}))  #featured channels
    except:
        channel_data['n_featured_channels'] = ''
    try:
        channel_data['n_links'] = len(contents_div.find_all('a', attrs={'class': 'about-channel-link'}))  #marketing/social media links 
    except:   
        channel_data['n_links'] = ''
    try:
        channel_data['description_length'] = len(contents_div.find('div', attrs={'class': 'about-description branded-page-box-padding'}).text.strip().replace('\n', '')) #description length
    except:   
        channel_data['description_length'] = ''
    try:
        channel_data['description'] = contents_div.find('div', attrs={'class': 'about-description branded-page-box-padding'}).text.strip().replace('\n', '')
    except:   
        channel_data['description'] = ''

    return channel_data

In [57]:
#yoga_channel is a list of yoga channel URLs
channels_data = defaultdict()

for channel in yoga_channel:
    try:
        channel_data = get_channel_info_about(channel)
        channels_data[channel] = channel_data
    except:
        continue
    
    time.sleep(.5+2*random.random())

In [58]:
len(channels_data)

607

In [59]:
with open('channels_about_607_att2.pkl', 'wb') as picklefile:
    pickle.dump(channels_data, picklefile)

## Scraping "Videos" page of yoga channels 

First n videos (~30 per page), without scrolling 

In [61]:
def get_videos_page_info(url):
    
    try:
        response = requests.get((url + '/videos'))
        page = response.text
        soup = BeautifulSoup(page, "html5lib")
        contents_div = soup.find('div', id='content')
    except:
        videos_info = {}
    
    try:
        titles = contents_div.find_all('a', attrs={'class': 'yt-uix-sessionlink yt-uix-tile-link spf-link yt-ui-ellipsis yt-ui-ellipsis-2'})
    except:
        videos_info = {}
    
    try:
        views_days = contents_div.find_all('ul', attrs={'class': 'yt-lockup-meta-info'})
    except:
        views_days = ''
    try:
        duration = contents_div.find_all('span', attrs={'class': 'accessible-description'})
    except:
        duration = ''

    videos_info = {}
    count = 1 
    for video in range(len(titles)):
        video_info = {}
        title = titles[video].get('title')
        views_age = views_days[video].text.strip()
        length = duration[video].text.strip()
    
        video_info['title'] = title
        video_info['views_days'] = views_age
        video_info['duration'] = length
        videos_info[count] = video_info
        count += 1
    
    return videos_info

In [62]:
#yoga_channel is a list of yoga channel URLs
videos_page_data = defaultdict()

for channel in yoga_channel:
    try:
        channel_data = get_videos_page_info(channel)
        videos_page_data[channel] = channel_data
    except:
        continue
    
    time.sleep(.5+2*random.random())

In [63]:
len(videos_page_data)

590

In [65]:
with open('Attempt2/channels_videos_page_590_att2.pkl', 'wb') as picklefile:
    pickle.dump(videos_page_data, picklefile)

## Getting links to videos by channel

In [77]:
def get_video_links_scroll(url):
    try:
        driver = webdriver.Chrome(chromedriver)
        driver.get((url + '/videos'))
        for i in range(45):
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        res = soup.find_all('a', attrs={'class': 'yt-simple-endpoint style-scope ytd-grid-video-renderer'}) #yt-uix-sessionlink 
        videos = []
        for i in res:
            videos.append(i.get('href').replace('/watch?v=', ''))
    except:
        videos = []
    return videos

In [78]:
#yoga_channel is a list of yoga channel URLs
videos_links_by_channel = defaultdict()

for channel in yoga_channel:
    try:
        channel_data = get_video_links_scroll(channel)
        videos_links_by_channel[channel] = channel_data
    except:
        continue
    
    time.sleep(.5+2*random.random())

In [79]:
with open('Attempt2/videos_links_by_channel_att2.pkl', 'wb') as picklefile:
    pickle.dump(videos_links_by_channel, picklefile)

## Getting datapoints by video

In [80]:
def get_video_info(url):
    
    video_data = {}
    try:
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "html5lib")
    except:
        video_data = {}
    
    try:
        video_data['Title'] = soup.find('span', attrs={'class': 'watch-title'}).text.strip()
    except:
        video_data['Title'] = ''
    try:
        video_data['Views'] = int(soup.find('div', attrs={'class': 'watch-view-count'}).text[:-6].replace(',', ''))
    except:
        video_data['Views'] = ''
    try:
        video_data['Description'] = soup.find('p', attrs={'id': 'eow-description'}).text
    except:
        video_data['Description'] = ''
    try:
        video_data['Publication Date'] = soup.find('strong', attrs={'class': 'watch-time-text'}).text.strip()
    except:
        video_data['Publication Date'] = ''
    try:
        video_data['Likes'] = int(soup.find('button', attrs={'title': 'I like this'}).text.replace(',', ''))
    except:
        video_data['Likes'] = ''
    try:
        video_data['Dislikes'] = int(soup.find('button', attrs={'title': 'I dislike this'}).text.replace(',', ''))
    except:
        video_data['Dislikes'] = ''
    
    return video_data

In [81]:
videos_info = defaultdict()
for key, val in links_dict.items():
    try:
        for i in val:
            url = 'https://www.youtube.com/watch?v=' + i
            videos_info[(key, i)] = get_video_info(url)
    except:
        continue 
        
    time.sleep(.5+2*random.random())

NameError: name 'links_dict' is not defined

In [ ]:
with open('Attempt2/videos_info_att2.pkl', 'wb') as picklefile:
    pickle.dump(videos_info, picklefile)

## Scraping comments with Selenium